# DOE Study Workflow

Asynchronous DOE Workflow with DOEStudy
=======================================

Demonstrates how to use DOEStudy to persist a Design of Experiments
workflow across multiple sessions. In practice, each "session" below
would be a separate script run (days or weeks apart) with lab work
in between.

The .jaxsr file is a ZIP archive containing:
- meta.json — version, timestamps
- study.json — factor config, design spec, model config, iteration history
- X_design.npy, X_observed.npy, y_observed.npy — NumPy binary arrays

In [1]:
import numpy as np

from jaxsr import DOEStudy

## Session 1: Set up the study and create the initial design

In [2]:
print("=" * 60)
print("SESSION 1: Study setup and initial design")
print("=" * 60)

study = DOEStudy(
    name="polymer_strength",
    description="Optimize tensile strength of a polymer blend",
    factor_names=["temperature", "pressure", "additive_pct"],
    bounds=[(150, 250), (5, 20), (0, 10)],
)

# Generate a space-filling design
X_design = study.create_design(method="latin_hypercube", n_points=15, random_state=42)
print(f"Created {len(X_design)} design points:")
for i, row in enumerate(X_design):
    print(f"  Run {i + 1}: T={row[0]:.1f}°C, P={row[1]:.1f} bar, additive={row[2]:.1f}%")

# Save and share with the lab
study.save("/tmp/polymer_study.jaxsr")
print(f"\nStudy saved. {len(study.pending_points)} experiments pending.")
print(study.summary())

SESSION 1: Study setup and initial design
Created 15 design points:
  Run 1: T=211.5°C, P=16.6 bar, additive=4.1%
  Run 2: T=198.7°C, P=6.9 bar, additive=6.0%
  Run 3: T=238.3°C, P=7.2 bar, additive=8.6%
  Run 4: T=233.7°C, P=15.6 bar, additive=3.4%
  Run 5: T=172.4°C, P=13.2 bar, additive=1.0%
  Run 6: T=168.5°C, P=5.4 bar, additive=6.0%
  Run 7: T=204.5°C, P=11.4 bar, additive=0.2%
  Run 8: T=227.6°C, P=8.0 bar, additive=2.7%
  Run 9: T=158.1°C, P=9.8 bar, additive=9.0%
  Run 10: T=156.4°C, P=14.8 bar, additive=1.5%
  Run 11: T=191.7°C, P=17.0 bar, additive=9.8%
  Run 12: T=187.5°C, P=10.5 bar, additive=2.5%
  Run 13: T=249.1°C, P=19.5 bar, additive=7.8%
  Run 14: T=178.9°C, P=12.6 bar, additive=4.8%
  Run 15: T=218.7°C, P=18.7 bar, additive=6.8%

Study saved. 15 experiments pending.
DOE Study: polymer_strength
Description: Optimize tensile strength of a polymer blend
Factors: temperature, pressure, additive_pct
Bounds: [(150, 250), (5, 20), (0, 10)]

Design: 15 points (0 completed, 

## Session 2: Add first batch of lab results and fit initial model

In [3]:
print("\n" + "=" * 60)
print("SESSION 2: First batch of results")
print("=" * 60)

# Reload from disk (simulating a new Python session)
study = DOEStudy.load("/tmp/polymer_study.jaxsr")
print(f"Loaded study: {study.n_observations} observations, {len(study.pending_points)} pending")

# Simulate lab results for the first 8 experiments
# (In practice, these come from real measurements)
X_batch1 = study.design_points[:8]


SESSION 2: First batch of results
Loaded study: 0 observations, 15 pending


In [4]:
def true_response(X):
    """Simulated ground truth for the polymer system."""
    T, P, A = X[:, 0], X[:, 1], X[:, 2]
    return 50 + 0.3 * T + 1.5 * P + 2.0 * A - 0.001 * T**2 + np.random.randn(len(T)) * 2

In [5]:
y_batch1 = true_response(X_batch1)

study.add_observations(X_batch1, y_batch1, notes="First 8 experiments from Lab A")
print(f"Added {len(y_batch1)} observations. Total: {study.n_observations}")

# Fit an initial model
model = study.fit(max_terms=5)
print(f"\nInitial model: {model.expression_}")
print(f"  MSE = {model._result.mse:.4f}")

# Save progress
study.save("/tmp/polymer_study.jaxsr")
print(f"\nStudy saved. {len(study.pending_points)} experiments still pending.")

Added 8 observations. Total: 8


/Users/jkitchin/Dropbox/projects/jaxsr/src/jaxsr/study.py:572: UserWarning: Removing 1 basis functions with non-finite values
  model.fit(self._X_observed, self._y_observed)



Initial model: y = 96.63 + 0.4055*pressure*additive_pct - 0.000363*temperature^2 - 2.908*log(additive_pct)
  MSE = 0.4076

Study saved. 7 experiments still pending.


## Session 3: Add remaining results and refine

In [6]:
print("\n" + "=" * 60)
print("SESSION 3: Complete results and refinement")
print("=" * 60)

study = DOEStudy.load("/tmp/polymer_study.jaxsr")
print(f"Loaded: {study.n_observations} observations, model fitted: {study.is_fitted}")
print(f"Current model: {study.model.expression_}")

# Add the remaining 7 experiments
X_batch2 = study.design_points[8:]
y_batch2 = true_response(X_batch2)
study.add_observations(X_batch2, y_batch2, notes="Remaining 7 experiments")

# Refit with all data
model = study.fit(max_terms=5)
print(f"\nRefined model: {model.expression_}")
print(f"  MSE = {model._result.mse:.4f}")

# Suggest next experiments for a follow-up round
next_points = study.suggest_next(n_points=3, strategy="space_filling")
print("\nSuggested next experiments:")
for i, row in enumerate(next_points):
    print(f"  Run {i + 1}: T={row[0]:.1f}°C, P={row[1]:.1f} bar, additive={row[2]:.1f}%")

study.save("/tmp/polymer_study.jaxsr")


SESSION 3: Complete results and refinement
Loaded: 8 observations, model fitted: True
Current model: y = 96.63 + 0.4055*pressure*additive_pct - 0.000363*temperature^2 - 2.908*log(additive_pct)



Refined model: y = 78.8 + 0.09344*pressure*additive_pct - 0.0002554*temperature^2 + 5.821*sqrt(pressure) + 0.1237*additive_pct^2
  MSE = 2.3617



Suggested next experiments:
  Run 1: T=169.7°C, P=19.6 bar, additive=5.8%
  Run 2: T=151.4°C, P=19.2 bar, additive=5.6%
  Run 3: T=151.2°C, P=5.6 bar, additive=0.0%


## Session 4: Review and share

In [7]:
print("\n" + "=" * 60)
print("SESSION 4: Review complete study")
print("=" * 60)

study = DOEStudy.load("/tmp/polymer_study.jaxsr")
print(study.summary())

# The .jaxsr file can be shared with colleagues
# They can load it and continue the analysis
print("\nFile /tmp/polymer_study.jaxsr is ready to share!")
print(f"  Schema version: {study.meta['schema_version']}")
print(f"  Created: {study.meta['created']}")
print(f"  Last modified: {study.meta['modified']}")


SESSION 4: Review complete study
DOE Study: polymer_strength
Description: Optimize tensile strength of a polymer blend
Factors: temperature, pressure, additive_pct
Bounds: [(150, 250), (5, 20), (0, 10)]

Design: 15 points (15 completed, 0 pending)
Design method: latin_hypercube
Observations: 15

Model: y = 78.8 + 0.09344*pressure*additive_pct - 0.0002554*temperature^2 + 5.821*sqrt(pressure) + 0.1237*additive_pct^2
  MSE: 2.36166
  AIC: 65.4586
  Terms: 5

Iterations: 2
  Round 1: +8 points → y = 96.63 + 0.4055*pressure*additive_pct - 0.000363*temperature^2 - 2.908*log(additive_pct) (First 8 experiments from Lab A)
  Round 2: +7 points → y = 78.8 + 0.09344*pressure*additive_pct - 0.0002554*temperature^2 + 5.821*sqrt(pressure) + 0.1237*additive_pct^2 (Remaining 7 experiments)

Created: 2026-02-07T12:02:09.641509+00:00
Modified: 2026-02-07T12:02:13.953031+00:00

File /tmp/polymer_study.jaxsr is ready to share!
  Schema version: 1.0.0
  Created: 2026-02-07T12:02:09.641509+00:00
  Last mod